In [1]:
! nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-8b98f8ad-ad55-4891-ea50-20783d5d4fab)


In [2]:
%%time

! pip install -qq -U langchain tiktoken pypdf faiss-gpu
! pip install -qq -U transformers InstructorEmbedding sentence_transformers
! pip install -qq -U accelerate bitsandbytes xformers einops
!pip install gtts

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.6.0 requires torch==2.0.0, but you have torch 2.0.1 which is incompatible.
CPU times: user 4.5 s, sys: 717 ms, total: 5.21 s
Wall time: 4min 16s


In [3]:
from gtts import gTTS   #we need it to create an audio files
import os   #we need it to save the audio files
import IPython   #we need it to listen the audio files

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print(langchain.__version__)

0.0.306


In [4]:
glob.glob('/kaggle/input/nasa-pdf/Nasa/*')

['/kaggle/input/nasa-pdf/Nasa/nasa-hdbk-873919-3.pdf',
 '/kaggle/input/nasa-pdf/Nasa/NASA-STD-6001B-w-Change-2_Admin-Change.pdf',
 '/kaggle/input/nasa-pdf/Nasa/nasa-hdbk-873919-4.pdf',
 '/kaggle/input/nasa-pdf/Nasa/nasa_std_7009a_change_1.pdf',
 '/kaggle/input/nasa-pdf/Nasa/nasa-std-873912_with_change_3.pdf',
 '/kaggle/input/nasa-pdf/Nasa/nasa-std-5018_revalidated (1).pdf',
 '/kaggle/input/nasa-pdf/Nasa/tb_21-04_copv_life_073021 (1).pdf',
 '/kaggle/input/nasa-pdf/Nasa/nesc-tb-20-08_10-1-20_state_of_hydrazine_synthesis.pdf',
 '/kaggle/input/nasa-pdf/Nasa/NASA-STD-4003A_w-Change-1-Revalidated.pdf',
 '/kaggle/input/nasa-pdf/Nasa/tb-21-01_051823.pdf',
 '/kaggle/input/nasa-pdf/Nasa/483905main_NESCTECHBUL10-02(Pyrovalve).pdf',
 '/kaggle/input/nasa-pdf/Nasa/nasa-hdbk-873921.pdf',
 '/kaggle/input/nasa-pdf/Nasa/tb_23-05_080123_1.pdf',
 '/kaggle/input/nasa-pdf/Nasa/nasa-hdbk-873923a.pdf',
 '/kaggle/input/nasa-pdf/Nasa/TB_22-08_Contaminant Reduction_083022-FINAL.pdf',
 '/kaggle/input/nasa-pdf/Nas

In [5]:
class CFG:
    # LLMs
    model_name = 'llama2-13b' # wizardlm, bloom, falcon, llama2-7b, llama2-13b
    temperature = 0
    top_p = 0.95
    repetition_penalty = 1.15

    # splitting
    split_chunk_size = 150
    split_overlap = 0

    # embeddings
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2'

    # similar passages
    k = 1
    # paths
    PDFs_path = '/kaggle/input/nasa-pdf/Nasa/'
    #Embeddings_path =  '/kaggle/input/faiss-hp-sentence-transformers'


In [6]:
def get_model(model = CFG.model_name):

    print('\nDownloading model: ', model, '\n\n')    

    model == 'llama2-13b'
    model_repo = 'daryl149/llama-2-13b-chat-hf'
        
    tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)

    model = AutoModelForCausalLM.from_pretrained(
        model_repo,
        load_in_4bit=True,
        device_map='auto',
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )
        #2048
    max_len = 204   

   
    return tokenizer, model, max_len

In [7]:
%%time
tokenizer, model, max_len = get_model(model = CFG.model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 44.2 s, sys: 1min 45s, total: 2min 30s
Wall time: 57min 26s


In [8]:
pipe = pipeline(
    task = "text-generation",
    model = model,
    tokenizer = tokenizer,
    pad_token_id = tokenizer.eos_token_id,
    max_length = max_len,
    temperature = CFG.temperature,
    top_p = CFG.top_p,
    repetition_penalty = CFG.repetition_penalty
)

llm = HuggingFacePipeline(pipeline = pipe)

In [17]:
query = "Which are the cardinal rules for safety software?"
llama = llm(query)

In [18]:
llama

'\n\nAnswer: There are several cardinal rules for ensuring safety in software development, including:\n\n1. Fail safe: The system should be designed to fail safely, meaning that if it fails, it will not cause harm or damage.\n2. Error handling: The system should be able to handle errors and exceptions gracefully, without causing harm or damage.\n3. Access control: The system should have appropriate access controls to prevent unauthorized access or modifications.\n4. Data validation: The system should validate all user input data to ensure that it is correct and valid.\n5. Secure coding practices: The system should be developed using secure coding practices, such as input validation, encryption, and secure communication protocols.\n6. Regular security testing: The system should be regularly tested for security vulnerabilities and penetration testing should be performed to identify any weaknesses.\n7. Secure storage of sensitive data: S'

In [19]:
audio_file = gTTS(text = llama, lang = "en", slow = False)

In [20]:
audio_file.save("audio.mp3")


In [21]:
file = "./audio.mp3"
IPython.display.display(IPython.display.Audio(file))

In [22]:
CFG.model_name

'llama2-13b'

In [23]:
%%time

loader = DirectoryLoader(
    CFG.PDFs_path,
    glob="./*.pdf",
    
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True
)

documents = loader.load()

100%|██████████| 166/166 [08:01<00:00,  2.90s/it]

CPU times: user 7min 59s, sys: 6.21 s, total: 8min 6s
Wall time: 8min 1s


In [24]:
documents[0]

Document(page_content=' \nAPPROVED FOR PUBLIC RELEASE —DISTRIBUTION IS U NLIMITED    NOT MEASUREMENT \nSENSITIVE  \n   \n  \nNASA TECHNICAL STANDARD  NASA -STD -7009 A \nw/ CHANGE 1: \nADMINISTRATIVE/  \nEDITORIAL CHANGES  \n2016 -12-07 \nNational Aeronautics and Space Administration  \n Approved:   2016 -07-13 \n Superseding NASA -STD -7009 A \n \n \n \n \n      \n \n \nSTANDARD FOR MODELS AND SIMULATIONS \n \n \n \n \n \n \n \n \n   \n ', metadata={'source': '/kaggle/input/nasa-pdf/Nasa/nasa_std_7009a_change_1.pdf', 'page': 0})

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CFG.split_chunk_size,
    chunk_overlap = CFG.split_overlap
)

texts = text_splitter.split_documents(documents)
len(texts)


184231

In [26]:
texts[0]

Document(page_content='APPROVED FOR PUBLIC RELEASE —DISTRIBUTION IS U NLIMITED    NOT MEASUREMENT \nSENSITIVE  \n   \n  \nNASA TECHNICAL STANDARD  NASA -STD -7009 A', metadata={'source': '/kaggle/input/nasa-pdf/Nasa/nasa_std_7009a_change_1.pdf', 'page': 0})

In [27]:
%%time

### download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
)

### load vector DB embeddings
#vectordb = FAISS.load_local(
#    CFG.Embeddings_path,
#    embeddings
#)

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 744 ms, sys: 357 ms, total: 1.1 s
Wall time: 2.77 s


In [28]:
db = FAISS.from_documents(texts, embeddings)

In [29]:
prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template = prompt_template, 
    input_variables = ["context", "question"]
)

In [30]:
retriever = db.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever, 
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)

In [32]:
question = "What is a General Interface Circuits for"
answer = db.max_marginal_relevance_search(question, k = CFG.k)

In [33]:
answer

[Document(page_content='4.2.5 General Interface Circuits (Command, Signal, Data, etc.) ...................... 16', metadata={'source': '/kaggle/input/nasa-pdf/Nasa/nasa-hdbk-4001.pdf', 'page': 4})]

In [35]:
### testing similarity search
question = "Uses of General Interface Circuits"
answer = db.similarity_search(question, k = CFG.k)

In [36]:
answer

[Document(page_content='It is desirable to make all explicit requirements testable.  To verify isolation of interface circuits,', metadata={'source': '/kaggle/input/nasa-pdf/Nasa/nasa-hdbk-4001.pdf', 'page': 13})]

In [41]:
def wrap_text_preserve_newlines(text, width=700):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    
    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4] + ' - page: ' + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )
    
    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [42]:
def llm_ans(query):
    start = time.time()
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

In [43]:
CFG.model_name

'llama2-13b'

In [44]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [46]:
query = "What is NASA-HDBK-4011?"
answers = (llm_ans(query))

In [50]:
audio_files = gTTS(text = answers, lang = "en", slow = False)


In [51]:
audio_files.save("audio2.mp3")



In [52]:
file = "./audio2.mp3"
IPython.display.display(IPython.display.|Audio(file))